In [ ]:
from astropy.io import fits
import pandas as pd
import numpy as np

def universal_fits_reader(file_path):
    """
    A robust reader that handles JWST, HST, and custom pipeline FITS files.
    """
    with fits.open(file_path) as hdul:
        # 1. Dynamically find the data extension
        # NASA usually puts data in 'EXTRACT1D', 'SCI', or Extension 1
        data_ext = None
        for i, hdu in enumerate(hdul):
            if hdu.data is not None:
                data_ext = hdu
                break
        
        if data_ext is None:
            raise ValueError("No tabular data found in FITS file.")

        df = pd.DataFrame(data_ext.data)
        cols = [c.upper() for c in df.columns]
        df.columns = cols

        # 2. Map Column Naming Variations
        # We look for keywords that imply Wavelength or Flux
        wave_variants = ['WAVELENGTH', 'WAVE', 'WAVELENGTH_UM', 'WAVELENGTH_ANGSTROMS']
        flux_variants = ['FLUX', 'RADIANCE', 'RADIANCE_FINAL', 'SURFACE_BRIGHTNESS']

        found_wave = next((c for c in cols if c in wave_variants), None)
        found_flux = next((c for c in cols if c in flux_variants), None)

        if not found_wave or not found_flux:
            # Fallback: Guess by position (0 is usually wavelength, 1 is flux)
            found_wave, found_flux = cols[0], cols[1]

        # 3. Standardize for the LLM
        clean_df = pd.DataFrame({
            'wavelength': df[found_wave],
            'flux': df[found_flux]
        })

        return clean_df, hdul[0].header

# --- Example Usage ---
# df_earth, head_earth = universal_fits_reader('earth_jwst_mimic_x1d.fits')
# df_jwst, head_jwst = universal_fits_reader('jw01248001001_03105_00001_mirifushort_x1d.fits')